In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
data = pd.read_csv("data/tagged_coral_condition_data_2.txt",sep = '\t')
data.head(5)

,date,reef,dir,channel,site,transect,coral,species,surveyor,BB,...,RMS,SA,SED,TO,UC,UD,UPM,WB,WP,<NA>
0,2013-10-20,HB,N,channelside,HBN1,1,HBN1.1.1,SSID,MLR,False,...,False,False,False,False,False,False,False,False,False,True
1,2013-10-20,HB,N,channelside,HBN1,1,HBN1.1.2,SSID,MLR,False,...,False,False,False,False,False,False,False,False,False,False
2,2013-10-20,HB,N,channelside,HBN1,1,HBN1.1.3,SSID,MLR,False,...,False,False,False,False,False,False,False,False,False,True
3,2013-10-20,HB,N,channelside,HBN1,1,HBN1.1.4,SSID,MLR,False,...,False,True,False,False,False,False,False,False,False,False
4,2013-10-20,HB,N,channelside,HBN1,1,HBN1.1.5,SBOU,MLR,False,...,False,False,False,False,False,False,False,False,False,True


#### Data rearangement
- Remove obs with NAs in condition column.
- Sort values by species.
- Select the coral species with at least one case of WP.
- Sort values by coral and survey date.


In [3]:
# sort values by species
data = data.sort_values(by = ['species'])


In [4]:
# remove NAs in condition column
data = data[data.iloc[:,-1] == False]
#data = data.reset_index(drop=True)


In [5]:
data

,date,reef,dir,channel,site,transect,coral,species,surveyor,BB,...,RMS,SA,SED,TO,UC,UD,UPM,WB,WP,<NA>
16920,2014-10-17,R2,S,control,R2SC2,2,R2SC2.2.4,AAGA,AS,False,...,False,False,True,False,False,False,False,False,False,False
13111,2014-08-21,R2,S,control,R2SC2,2,R2SC2.2.4,AAGA,RF,False,...,False,False,False,False,False,False,False,False,False,False
19473,2015-03-17,R2,S,control,R2SC2,2,R2SC2.2.4,AAGA,AS,False,...,False,False,False,False,False,False,True,False,False,False
22166,2015-07-07,R2,S,control,R2SC2,2,R2SC2.2.4,AAGA,RF,False,...,False,False,False,False,False,False,False,False,False,False
10955,2014-07-17,R2,S,control,R2SC2,2,R2SC2.2.4,AAGA,RF,False,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6209,2014-05-30,HB,N,channelside,HBN2,1,HBN2.1.5,SSID,MLR,False,...,False,True,False,False,False,False,False,False,False,False
6204,2014-05-29,HB,N,channelside,HBN2,3,HBN2.3.6,SSID,RF,False,...,False,False,False,False,False,False,False,False,False,False
6202,2014-05-29,HB,N,channelside,HBN2,3,HBN2.3.4,SSID,RF,False,...,False,False,False,False,False,False,False,False,False,False
6201,2014-05-29,HB,N,channelside,HBN2,3,HBN2.3.3,SSID,RF,False,...,False,True,False,False,False,False,False,False,False,False


In [6]:
# create an index representing the row position of the first coral of a unique species in the dataset
temp = data.duplicated(subset = ["species"])
coral_species_index = np.arange(data.shape[0])[~temp]

# create a fake species index, representing the end
coral_species_index = np.append(coral_species_index,data.shape[0])



# append the coral species which experienced WP into a set
coral_species_index_of_interest = set()
for i in range(len(coral_species_index)-1):
    
    temp= data.WP.iloc[coral_species_index[i]:coral_species_index[i+1]]  
    #if np.any(temp):
    if np.sum(temp) > 1:
        coral_species_index_of_interest.add(coral_species_index[i])


coral_species_of_interest = [data.species.values[i] for i in coral_species_index_of_interest]
coral_species_of_interest

# select the coral whose species is of interest
data = data[data.species.isin(coral_species_of_interest)]

# sort the data by coral name and survey date
data = data.sort_values(by = ['coral','date'])
#data = data.reset_index(drop=True)

#### Create a state column

- If the coral is not WP,PM/DEAD, the state is "Healthy".
- If WP == True, the state is "WP".
- If PM/DEAD == True, the state is "PM/DEAD".


In [7]:
state = []
for i in range(data.shape[0]):
    if data.DEAD.iloc[i] == True:
        #state.append("DEAD")
        #state.append(4)
        state.append(3)
        
    elif data.PM.iloc[i] == True:
        #state.append("PM")
        state.append(3)
    elif data.WP.iloc[i] == True:
        #state.append("WP")
        state.append(2)
    else:
        #state.append("HEALTHY")
        state.append(1)
data["state"] = state
data.head(10)       

,date,reef,dir,channel,site,transect,coral,species,surveyor,BB,...,SA,SED,TO,UC,UD,UPM,WB,WP,<NA>,state
50,2013-11-01,HB,N,channelside,HBN1,1,HBN1.1.5,SBOU,WFP,False,...,False,False,False,False,False,False,False,False,False,1
73,2013-11-07,HB,N,channelside,HBN1,1,HBN1.1.5,SBOU,MLR,False,...,False,False,False,False,False,False,False,False,False,1
2210,2013-11-25,HB,N,channelside,HBN1,1,HBN1.1.5,SBOU,JDC,False,...,False,False,False,False,False,False,False,False,False,1
14192,2014-09-12,HB,N,channelside,HBN1,1,HBN1.1.5,SBOU,AS,False,...,False,False,False,False,False,False,False,False,False,1
15187,2014-09-22,HB,N,channelside,HBN1,1,HBN1.1.5,SBOU,RF,False,...,False,False,False,False,False,False,False,False,False,1
20655,2015-06-23,HB,N,channelside,HBN1,1,HBN1.1.5,SBOU,RF,False,...,False,False,False,False,False,False,False,False,False,1
21214,2015-06-30,HB,N,channelside,HBN1,1,HBN1.1.5,SBOU,RF,False,...,True,False,False,False,False,False,False,False,False,1
21797,2015-07-09,HB,N,channelside,HBN1,1,HBN1.1.5,SBOU,RF,False,...,False,False,False,False,False,False,False,False,False,1
22444,2015-07-13,HB,N,channelside,HBN1,1,HBN1.1.5,SBOU,RF,False,...,False,False,False,False,False,False,False,False,False,1
9,2013-10-20,HB,N,channelside,HBN1,2,HBN1.2.1,SBOU,MLR,False,...,False,False,False,False,False,False,False,False,False,1


#### Observe the transition patterns
- Examine if there are abnormal transitions.
- If the coral is PM/DEAD, we assume no transition will happend onwards.

In [8]:
temp = data.duplicated(subset = ["coral"])
coral_index = np.arange(data.shape[0])[~temp]
coral_index = np.append(coral_index,data.shape[0])

In [9]:
#abnormal_coral_index = []
abnormal_obs_mask = np.zeros(data.shape[0],dtype = bool)
for i in range(len(coral_index)-1):
    temp = data.state.iloc[coral_index[i]:coral_index[i+1]]
    flag_DEAD = 0
    for j in range(len(temp)):
        # if DEAD occurs, the left will be DEAD onwards
        #if j == "DEAD":
        if flag_DEAD:
            #if j != "DEAD":
            #if temp.iloc[j] != 3:
            #abnormal_coral_index.append(coral_index[i])
            abnormal_obs_mask[(coral_index[i]+j):coral_index[i+1]] = 1
            break
        if temp.iloc[j] == 3:
            flag_DEAD = 1
        

In [10]:
# remove the abnormal corals from the dataset
#abnormal_coral = data.coral.iloc[abnormal_coral_index]
#data = data[~data.coral.isin(abnormal_coral)]    
#data = data.reset_index(drop=True)

# we are throwing away many obs here!
data = data[~abnormal_obs_mask]
data = data.reset_index(drop=True)

#### Add several columns used as covariates
- bleaching, buried.
- control site.
- survey date.


In [11]:
bur_coral = []
bl_coral = []
for i in range(data.shape[0]):
    if data.BL.iloc[i] or data.PB.iloc[i]:
        bl_coral.append(1)
    else:
        bl_coral.append(0)
    if data.BUR.iloc[i] or data.PBUR.iloc[i]:
        bur_coral.append(1)
    else:
        bur_coral.append(0)
        
data['bl_coral'] = bl_coral
data['bur_coral'] = bur_coral

# add control site covariate
data['control'] = 0
for i in range(data.shape[0]):
    if 'C' in data.site.iloc[i]:
        data.control.iloc[i] = 1

/Users/kehuiyao/Desktop/coral/venv/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [12]:
data.date = pd.to_datetime(data.date)


In [13]:
months = []
temp = data.duplicated(subset = ["coral"])
coral_index = np.arange(data.shape[0])[~temp]
coral_index = np.append(coral_index,data.shape[0])


for i in range(len(coral_index)-1):
    temp = data.date.iloc[coral_index[i]:coral_index[i+1]]
    start_date = data.date.iloc[coral_index[i]]
    for j in temp:
        months.append((j - start_date)/np.timedelta64(1, 'M'))
data['months'] = months

#### Make an assumption: if a coral is buried or bleaching at some time, it never recovers in its lifetime in the survey

In [14]:
# temp = data.duplicated(subset = ["coral"])
# coral_index = np.arange(data.shape[0])[~temp]
# coral_index = np.append(coral_index,data.shape[0])

# for i in range(len(coral_index)-1):
#     for j in range(coral_index[i],coral_index[i+1]):
#         if data.bur_coral.iloc[j] == 1:
#             data.bur_coral.iloc[(j+1):coral_index[i+1]] = 1
#             break
#     for j in range(coral_index[i],coral_index[i+1]):
#         if data.bl_coral.iloc[j] == 1:
#             data.bl_coral[(j+1):coral_index[i+1]] = 1
#             break


# if there is one case of buried, set bur_coral = 1

# temp = data.duplicated(subset = ["coral"])
# coral_index = np.arange(data.shape[0])[~temp]
# coral_index = np.append(coral_index,data.shape[0])

# for i in range(len(coral_index)-1):
#     for j in range(coral_index[i],coral_index[i+1]):
#         if data.bur_coral.iloc[j] == 1:
#             data.bur_coral.iloc[coral_index[i]:coral_index[i+1]] = 1
#             break
#     for j in range(coral_index[i],coral_index[i+1]):
#         if data.bl_coral.iloc[j] == 1:
#             data.bl_coral[coral_index[i]:coral_index[i+1]] = 1
#             break

#### How many buried observations and bleaching observations in total

In [15]:
print("Total number of bleaching obs is %d"%np.sum(data.bl_coral))
print("Total number of buried obs is %d"%np.sum(data.bur_coral))

Total number of bleaching obs is 431
Total number of buried obs is 1175


#### Time distribution when some transition happens
- Healthy to WP
- Healthy to DEAD

In [27]:
def find_transition_a_to_b(state, start, end):
    res = []
    i = 0
    while i < len(state)-1:
        if state[i] == start:
            if state[i+1] == end:
                #res.append(i)
                res.append(i+1)
                i += 1
        i += 1
    return res

# distribution of the time when transition 1-2 happens
time_transition_1_to_2 = data.iloc[find_transition_a_to_b(data.state,1,2),:].months
#px.histogram(time_transition_1_to_2,labels={'value':'time (months)'},title = "Transition from Healthy to WP")
px.histogram(time_transition_1_to_2,labels={'value':'time (months)'})
# distribution of the time when transition 1-3 happens
time_transition_1_to_3 = data.iloc[find_transition_a_to_b(data.state,1,3),:].months
#px.histogram(time_transition_1_to_3,labels={'value':'time (months)'},title = "Transition from Healthy to PM/DEAD")
px.histogram(time_transition_1_to_3,labels={'value':'time (months)'})

# # distribution of the time when transition 2-1 happens
# time_transition_2_to_1 = data.iloc[find_transition_a_to_b(data.state,2,1),:].months
# px.histogram(time_transition_2_to_1)

                

#### Save the cleaned version of the dataset

In [17]:
data = data.reset_index(drop=True)
data.to_csv("data_cleaned.csv")

In [18]:
print("number of species is %d"%len(data.species.unique()))
print("they are ",data.species.unique())
print("number of corals include in the study is %d"%len(data.coral.unique()))
print("total number of obs is %d"%data.shape[0])

number of species is 8
they are  ['SBOU' 'DSTO' 'DCLI' 'PSTR' 'MCAV' 'MMEA' 'CNAT' 'MDEC']
number of corals include in the study is 378
total number of obs is 9259


#### If it does not converge, here is some tricks to simplify the data

In [19]:
# first = data.duplicated(subset = ["coral","state"],keep ='first')
# last = data.duplicated(subset = ["coral","state"],keep ='last')
# data_simplified = data[(~first) | (~last)]




In [20]:
# data_simplified.to_csv("data_cleaned_simplified.csv")